#Reading the File

In [ ]:
import pandas as pd 
from pandas import read_excel 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

my_sheet = 'Sheet6'
file_name = 'Record_Colab.xlsx'
data = read_excel(file_name, sheet_name = my_sheet)
#print(data.head())

We have continuous features only. 

In [ ]:
#Removing few irrelevant features 
data=data.drop("ID", axis=1)
data=data.drop("SMU_total", axis=1)
data=data.drop("SMU_available", axis=1)
data=data.drop("DU_total", axis=1)
data=data.drop("DU_used", axis=1)
data=data.drop("DU_free", axis=1)


continuous_features = [col for col in data.columns if data[col].dtype != 'O']

#continuous_features.remove('ID')
continuous_features.remove('MGUP') # remove the target feature (Change to MGUP/MGMAP)
#categorical_features = continuous_features


#Coefficient of Determination 

We derive the coefficient of determination,  r2 , which is based on Pearson's correlation coefficient  r , for each feature with respect to 'MGUP'. The higher  r2 , the higher the correlation between the two features.

The coefficient of determination is typically considered a measure for the variance in a relation between two features that is explained by the model and and does not rely on any machine learning model. 

Change pearson to kendall, spearman for those correlaions. 

In [ ]:
#f, (ax1, ax2) = plt.subplots(2, 1, figsize=(40, 25), sharey=True)
f, ax1 = plt.subplots(figsize=(17, 5))
#plt.subplots_adjust(hspace=0.5)

# continuous features
data[continuous_features].corrwith(data.MGUP, method='pearson').agg('square').plot.bar(ax=ax1, alpha=1.0) #(Change to MGUP/MGMAP)
ax1.set_title('Coefficient of Determination for Max. GPU Utilization Percent: Pearson Method')
ax1.grid()
ax1.set_xlabel("Features", fontsize=30)
ax1.set_ylabel("Coefficients")

Important Features: 

In [ ]:
results = pd.DataFrame(data.drop('MGMAP', axis=1).corrwith(data.MGMAP).agg('square'), #(Change to MGUP/MGMAP)
                       columns=['det_weight'])

ranks = np.zeros(len(results), dtype=np.int)
for i, j in enumerate(np.argsort(results.det_weight)[::-1]):
    ranks[j] = i
results['det_rank'] = ranks

results.sort_values('det_rank').loc[:, ['det_rank', 'det_weight']].iloc[0:60]

results_trimmed = results[(results['det_weight'] >0.5)]

results_trimmed.sort_values('det_weight', ascending=False)

#Random Forest Feature Importance


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

pipe = Pipeline([('model', RandomForestRegressor(n_jobs=-1, random_state=42))])

param_space = {'model__max_depth': [3, 4, 6, 8, 10, 15, 20],
               'model__max_features': [3, 4, 6, 8, 10, 15, 20],
               'model__n_estimators': [50, 100, 150]}

grid = GridSearchCV(pipe, param_grid=param_space, cv=10, scoring='neg_mean_squared_error') 

grid.fit(data.drop('MGMAP', axis=1), data.MGMAP) #(Change to MGUP/MGMAP)


In [ ]:
print('best-fit parameters:', grid.best_params_)
print('prediction rms residuals:', 10**(np.sqrt(-grid.best_score_)))

Identifying the important features

In [ ]:
weights = grid.best_estimator_.steps[0][1].feature_importances_

results['rf_weight'] = weights

ranks = np.zeros(len(results), dtype=np.int)
for i, j in enumerate(np.argsort(weights)[::-1]):
    ranks[j] = i
results['rf_rank'] = ranks

results.sort_values('rf_rank').loc[:, ['rf_rank', 'rf_weight']].iloc[0:100]

#Linear Model Coefficient Weights


#Ridge Regression


In [ ]:
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import Ridge

pipe = Pipeline([('scaler', RobustScaler()), 
                 ('model', Ridge(random_state=42))])

param_space = {'model__alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}

grid = GridSearchCV(pipe, param_grid=param_space, cv=10, scoring='neg_mean_squared_error')

grid.fit(data.drop('MGMAP', axis=1), data.MGMAP) #(Change to MGUP/MGMAP)

In [ ]:
print('best-fit parameters:', grid.best_params_)
print('prediction rms residuals:', 10**(np.sqrt(-grid.best_score_)))

Most important features are

In [ ]:
weights = grid.best_estimator_.steps[1][1].coef_

results['ridge_weight'] = weights

ranks = np.zeros(len(results), dtype=np.int)
for i, j in enumerate(np.argsort(weights)[::-1]):
    ranks[j] = i
results['ridge_rank'] = ranks

results.sort_values('ridge_rank').loc[:, ['ridge_rank', 'ridge_weight']].iloc[0:100]

#Lasso


In [ ]:
from sklearn.linear_model import Lasso

pipe = Pipeline([('scaler', RobustScaler()), 
                 ('model', Lasso(random_state=42))])

param_space = {'model__alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}

grid = GridSearchCV(pipe, param_grid=param_space, cv=10, scoring='neg_mean_squared_error')

grid.fit(data.drop('MGMAP', axis=1), data.MGMAP) #(Change to MGUP/MGMAP)

In [ ]:
print('best-fit parameters:', grid.best_params_)
print('prediction rms residuals:', 10**(np.sqrt(-grid.best_score_)))

Important features are:

In [ ]:
weights = grid.best_estimator_.steps[1][1].coef_

results['lasso_weight'] = weights

ranks = np.zeros(len(results), dtype=np.int)
for i, j in enumerate(np.argsort(weights)[::-1]):
    ranks[j] = i
results['lasso_rank'] = ranks

results.sort_values('lasso_rank').loc[:, ['lasso_rank', 'lasso_weight']].iloc[0:100]

#Comparison of Results


In [ ]:
f, ax = plt.subplots(figsize=(17, 5))

results['mean_rank'] = results.loc[:, ['det_rank', 'rf_rank', 'ridge_rank', 'lasso_rank']].agg('abs').mean(axis=1)
results['mean_rank_std'] = results.loc[:, ['det_rank', 'rf_rank', 'ridge_rank', 'lasso_rank']].std(axis=1)

results.mean_rank.sort_values().plot.bar(width=0.5, color='orange', alpha=1.0, ax=ax)
results.sort_values(by='mean_rank').mean_rank_std.plot.bar(width=0.5, color='blue', alpha=0.5, ax=ax)

ax.set_xlim([0,59])
ax.plot(ax.get_xlim(), [0, 59], color='red')
ax.set_ylabel('Average Rank')
ax.set_xlabel("Ranks")
ax.set_ylabel("Features")

Orange bars indicate the mean rank - blue bars indicate the standard deviation of the mean rank across the four methods. The red line symbolizes ideal behavior and unity line: the rank increases by one for each additional feature.

The following plot visualizes the same data in a different way:

In [ ]:
from matplotlib import ticker

f, ax = plt.subplots(figsize=(5, 17))

for idx, feature in enumerate(results.sort_values(by='mean_rank').index):
    ax.plot(range(4), results.loc[feature, ['det_rank', 'rf_rank', 'ridge_rank', 'lasso_rank']]+0.5, marker='o',
           alpha=np.clip(1/(results.loc[feature, 'mean_rank_std']+0.001), 0.01, 1),
           linewidth=np.clip(1/(results.loc[feature, 'mean_rank_std']+0.1), 0.5, 1)*3)
ax.set_ylim([59, 0])
ax.yaxis.set_major_locator(ticker.LinearLocator(numticks=49))
ax.set_yticklabels(results.sort_values(by='mean_rank').index)
ax.xaxis.set_major_locator(ticker.LinearLocator(numticks=4))
ax.set_xticklabels(['Coeff_of_Det', 'Random_Forest', 'Ridge_Regression', 'Lasso_Regression'])
ax.set_xlabel("Ranks")
ax.set_ylabel("Features")

In [ ]:
for col in ['det_weight', 'rf_weight', 'ridge_weight', 'lasso_weight']:
    weightsum = results.loc[:, col].abs().sum(axis=0)
    results.loc[:, col] = results.loc[:, col].apply(lambda x: np.abs(x)/weightsum)

Now we can plot a cumulative distribution of the weights derived from the different methods:

In [ ]:
f, ax = plt.subplots(figsize=(17, 5))
ax.xaxis.set_major_locator(ticker.LinearLocator(numticks=18))
results.sort_values('mean_rank').loc[:, ['det_weight', 'rf_weight', 'ridge_weight', 'lasso_weight']].cumsum().plot.line(
    rot='vertical', ax=ax)
ax.grid()

In [ ]:
#Using Correlation Matrix 
plt.figure(figsize=(25,10))
cor = data.corr()
sns.heatmap(cor, annot=False, cmap=plt.cm.Reds)
plt.show()